# Deep Computer Vision Using Convolutional Neural Networks

## Implementing Convolutional Layers with Keras

In [1]:
## Get some sample images
from sklearn.datasets import load_sample_images
import tensorflow as tf

images = load_sample_images()['images']
images = tf.keras.layers.CenterCrop(height=70, width=120)(images)
images = tf.keras.layers.Rescaling(scale=1 / 255)(images)

2025-01-12 10:59:39.127422: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-12 10:59:39.307052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736701179.371375    4119 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736701179.390691    4119 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 10:59:39.569730: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
images.shape
## 4d tensor
## 2 Images in tensor
## 70 is the height in pixels
## 120 is the width in pixels
## 3 is the amount of color channels(most likly RGB)

TensorShape([2, 70, 120, 3])

In [4]:
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=7)
fmaps = conv_layer(images)

I0000 00:00:1736701339.060876    4119 cuda_dnn.cc:529] Loaded cuDNN version 90300


In [7]:
fmaps.shape
## 32 channels (because of the filters)
## The height and width shrunk because Conv2d doesn't zero pad by defualt
## 2 images

TensorShape([2, 64, 114, 32])

In [8]:
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=7,
                                   padding='same') ## Zero padding
fmaps = conv_layer(images)

In [10]:
fmaps.shape

TensorShape([2, 70, 120, 32])

In [15]:
kernels, biases = conv_layer.get_weights()
biases.shape
## 1d shape [output_channels]

(32,)

In [16]:
kernels.shape
## 4d shape [kernel_height, kernel_width, input_channels, output_channels]

(7, 7, 3, 32)

<b>You should also add activation functions and kernel_initilizers to Convolutional layers, for the same reasons as Dense layers

## Pooling Layers

Pooling layers are ment to subsample(i.e. shrink) the input image in order to reduce the computational load, the memory usage, and the number of parameters(thereby limiting the risk of overfitting)